### <span style="color:red">CONFIGURACIÓN: EJECUTAR UNA SOLA VEZ CUANDO SE INICIA EL COLAB.</span>

In [ ]:
!git clone https://github.com/mayala-afi/econometria-mf.git

%cd econometria-mf

!pip install -q -r requirements.txt

## SESION 3 : MODELOS VAR/VEC

#### Importación de librerías

In [ ]:
import pandas as pd
import numpy as np

from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.vecm import coint_johansen, VECM
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA


from utils import clean_fred_df, adf_test, irf_plot, plot_forecast, plot_acf_pacf, predict_plot_multi

import warnings

warnings.filterwarnings("ignore")

### 1. Importar datos desde Excel descargado de FRED

In [ ]:
cpi_df = pd.read_excel("./data/CPIAUCSL.xlsx", sheet_name="Monthly")
ff_df = pd.read_excel("./data/FEDFUNDS.xlsx", sheet_name="Monthly")
pib_df = pd.read_excel("./data/GDPC1.xlsx", sheet_name="Quarterly")

### 2. Limpieza y preparación de datos

#### CPI (mensual --> trimestral promedio) 

In [ ]:
cpi_df = clean_fred_df(cpi_df, "CPIAUCSL", "cpi")
cpi_q = (
    cpi_df["cpi"].to_period("M").to_timestamp().resample("Q").mean().to_period("Q")
)

#### FEDFUNDS (mensual --> trimestral promedio) 

In [ ]:
ff_df = clean_fred_df(ff_df, "FEDFUNDS", "ff")
ff_q = ff_df["ff"].to_period("M").to_timestamp().resample("Q").mean().to_period("Q")

#### PIB (ya trimestral)

In [ ]:
gdp_df = clean_fred_df(pib_df, "GDPC1", "pib")
gdp_q = gdp_df["pib"].to_period("Q")

#### Combinación de las series en un solo DataFrame

In [ ]:
df = pd.concat([gdp_q, cpi_q, ff_q], axis=1)

df = df.loc["1988-10-01":]
display(df.head())

### 3. Transformaciones

In [ ]:
df["lnpib"] = np.log(df["pib"])
df["lninflacion"] = np.log(df["cpi"])
df["inflacion"] = 100 * df["lninflacion"].diff()
df["tipo_interes"] = df["ff"]

df["dlnpib"] = df["lnpib"].diff()
df["dinflacion"] = df["inflacion"].diff()
df["dtipo_interes"] = df["tipo_interes"].diff()

df = df.dropna()

### 4. Pruebas ADF - raíz unitaria

In [ ]:
adf_test(df["lnpib"], "lnpib")
adf_test(df["inflacion"], "inflacion")
adf_test(df["tipo_interes"], "tipo_interes")

adf_test(df["dlnpib"], "dlnpib")
adf_test(df["dinflacion"], "dinflacion")
adf_test(df["dtipo_interes"], "dtipo_interes")

### 5. Selección de retardos y VAR

In [ ]:
var_data = df[["dlnpib", "inflacion", "dtipo_interes"]]
model = VAR(var_data)

model.select_order(maxlags=4)

In [ ]:
var_res = model.fit(1)
print(var_res.summary())

### 6. Causalidad de Granger

In [ ]:
var_res.test_causality("dlnpib", ["inflacion"], kind="f").summary()


### 7. Funciones Impulso-Respuesta

In [ ]:
irf = var_res.irf(12)

irf_plot(impulse="dtipo_interes", response="dlnpib", irf=irf)
irf_plot(impulse="inflacion", response="dlnpib", irf=irf)


### 8. Cointegración (Johansen)

In [ ]:
coint = coint_johansen(df[["lnpib", "inflacion", "tipo_interes"]], det_order=0, k_ar_diff=2)

coint.lr1  # estadísticos traza
coint.cvt  # valores críticos

_Si hay cointegración, entonces estimar un modelo VECM_

In [ ]:
vecm = VECM(df[["lnpib", "inflacion", "tipo_interes"]], k_ar_diff=2, coint_rank=1)

vecm_res = vecm.fit()
print(vecm_res.summary())


### 9. Previsiones VEC

In [ ]:
forecast = vecm_res.predict(steps=8)

_Dataframe para graficar_

In [ ]:
h = 8 
last_period = df.index[-1]
future_index = pd.period_range(start=last_period + 1, periods=h, freq="Q")
df_forecast = pd.DataFrame(
    forecast, index=future_index, columns=["lnpib", "inflacion", "tipo_interes"]
)

ln(PIB)

In [ ]:
print("Gráficos de previsiones VEC:")
print("PIB")
plot_forecast(df, df_forecast, "lnpib", "Previsión VEC - ln(PIB)")
print("Inflación")
plot_forecast(df, df_forecast, "inflacion", "Previsión VEC - Inflación")
print("Tipo de interés")
plot_forecast(df, df_forecast, "tipo_interes", "Previsión VEC - Tipo de interés")


In [ ]:
plot_forecast(df, df_forecast, "lnpib", "Previsión VEC - ln(PIB)")

### 10. Comparación del PIB: MCO vs VEC vs ARIMA

#### MCO: Regresión contemporánea (SESIÓN 1)

In [ ]:
Y = df["dlnpib"]
X = sm.add_constant(df[["inflacion", "dtipo_interes"]])

ols = sm.OLS(Y, X).fit()
df["yhat_mco"] = ols.fittedvalues

#### ARIMA Univariante (SESIÓN 2)

_Visualizar ACF y PACF para identificar órdenes_

In [ ]:
plot_acf_pacf(df, var="dlnpib", lags=12, title_suffix="Residuos MCO dlnpib")

In [ ]:
arima = ARIMA(df["dlnpib"], order=(1, 0, 1)).fit()
df["yhat_arima"] = arima.fittedvalues


#### VEC (SESIÓN 3)

In [ ]:
fitted = vecm_res.fittedvalues

fitted_df = pd.DataFrame(
    fitted, index=df.index[-fitted.shape[0] :], columns=["lnpib", "inflacion", "tasa"]
)

df["yhat_vec_lnpib"] = fitted_df["lnpib"]

df["yhat_vec"] = fitted_df["lnpib"].diff()


In [ ]:
df_plot = df.loc["2018-01-01":, ["dlnpib", "yhat_mco", "yhat_vec", "yhat_arima"]]

predict_plot_multi(
    df_plot,
    title="Comparación de modelos: dlnpib",
    labels=["Observado", "MCO", "VEC", "ARIMA"],
)

### 11. Errores cuadráticos

In [ ]:
df["error2_mco"] = (df["dlnpib"] - df["yhat_mco"]) ** 2
df["error2_vec"] = (df["dlnpib"] - df["yhat_vec"]) ** 2
df["error2_arima"] = (df["dlnpib"] - df["yhat_arima"]) ** 2

df[["error2_mco", "error2_vec", "error2_arima"]].mean()